In [1]:
import sendgrid
sg = sendgrid.SendGridAPIClient('API code')
import pandas as pd
import numpy as np
import re

In [2]:
import datetime
# end_date=datetime.date.today()
end_date='2022-03-30'
response = sg.client.marketing.singlesends.get(query_params={'start_date': '2021-01-01', 'end_date': end_date})
single_send=response.to_dict
single_send.keys()
single_send_list=pd.DataFrame.from_dict(single_send['result'])
single_send_list=single_send_list.loc[(~single_send_list['name'].apply(lambda x:re.match(r'((.*)(t|T)est(.*))|(Untitled Single Send)',x)).astype('bool'))&(single_send_list['status']=='triggered')]
single_send_list['uptime']=single_send_list.updated_at.apply(lambda x:datetime.datetime.strptime(re.findall(r'\d{4}-\d{2}-\d{2}',x)[0],'%Y-%m-%d'))
single_send_list=single_send_list.loc[single_send_list.uptime<datetime.datetime.strptime(end_date,'%Y-%m-%d'),]

In [3]:
single_send_list.head()

,id,name,status,categories,send_at,created_at,updated_at,is_abtest,abtest,uptime
17,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,[Email Blast - Enhance your customers mobile e...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,False,None,2022-03-23
18,630991dc-a940-11ec-9764-06ffdaa21214,03222022-AWS-Newsletter2,triggered,[Email Blast - Enhance your customers mobile e...,2022-03-22T18:00:00Z,2022-03-21T17:57:35Z,2022-03-22T18:00:34Z,False,None,2022-03-22
19,1acd5fc5-a61d-11ec-a2f0-76f7ae4699b7,03182022-Kaleidoscope-Learning-EBlast,triggered,[Email Blast - Kaleidoscope Learning],2022-03-18T22:25:49Z,2022-03-17T18:07:28Z,2022-03-18T22:26:08Z,False,None,2022-03-18
20,4685511e-a554-11ec-a098-527f968770fa,03172022_AtScale_E-Blast2,triggered,[Email Blast - How to Create & Foster a Data-D...,2022-03-17T18:00:00Z,2022-03-16T18:09:52Z,2022-03-17T18:00:23Z,False,None,2022-03-17
21,381ac770-9f34-11ec-94c1-429e7d595f19,03152022-AWS-Newsletter1,triggered,[Email Blast - Enhance your customers mobile e...,2022-03-15T22:32:27Z,2022-03-08T23:05:17Z,2022-03-15T22:33:09Z,False,None,2022-03-15


In [4]:
id_list=single_send_list.id.unique()
day_id_metric=pd.DataFrame()
for id in id_list:
    response = sg.client.marketing.stats.singlesends._(id).get(query_params={'aggregated_by':'day','start_date': '2021-01-01', 'end_date': end_date})
    stats=pd.DataFrame.from_dict(response.to_dict['results'])
    metric_df=pd.DataFrame()
    for i in range(len(stats)):
        tmp=stats['stats'][i]
        metric_df=metric_df.append(tmp,ignore_index=True)
    id_metric=pd.concat([stats,metric_df],axis=1)
    day_id_metric=day_id_metric.append(id_metric)
day_id_metric.head()
# pd.DataFrame.from_dict(tmp)
# for i in range(len(stats)):
#     tmp=stats.stats[i]
#     tmp=tmp[0]['metrics']
#     stats.stats[i]=tmp
# bdf=stats.stats.apply(pd.Series)
# stats=pd.concat([stats.date,bdf],axis=1)

,ab_phase,ab_variation,aggregation,stats,id,bounce_drops,bounces,clicks,delivered,invalid_emails,opens,requests,spam_report_drops,spam_reports,unique_clicks,unique_opens,unsubscribes
0,all,all,2022-03-23,"{'bounce_drops': 4328, 'bounces': 2834, 'click...",1cddb357-aad2-11ec-8d85-b2a3dfe36627,4328.0,2834.0,6788.0,20125.0,18.0,2122.0,28519.0,151.0,1.0,1049.0,1633.0,23.0
1,all,all,2022-03-24,"{'bounce_drops': 0, 'bounces': 9, 'clicks': 47...",1cddb357-aad2-11ec-8d85-b2a3dfe36627,0.0,9.0,47.0,977.0,0.0,2433.0,0.0,0.0,0.0,11.0,1721.0,20.0
2,all,all,2022-03-25,"{'bounce_drops': 0, 'bounces': 0, 'clicks': 0,...",1cddb357-aad2-11ec-8d85-b2a3dfe36627,0.0,0.0,0.0,17.0,0.0,290.0,0.0,0.0,1.0,0.0,175.0,2.0
3,all,all,2022-03-26,"{'bounce_drops': 0, 'bounces': 56, 'clicks': 4...",1cddb357-aad2-11ec-8d85-b2a3dfe36627,0.0,56.0,42.0,4.0,0.0,97.0,0.0,0.0,0.0,4.0,53.0,3.0
4,all,all,2022-03-27,"{'bounce_drops': 0, 'bounces': 0, 'clicks': 0,...",1cddb357-aad2-11ec-8d85-b2a3dfe36627,0.0,0.0,0.0,0.0,0.0,79.0,0.0,0.0,0.0,0.0,48.0,0.0


In [5]:
day_id_metric=pd.merge(day_id_metric,single_send_list,on='id',how='left')
day_id_metric.columns
day_id_metric=day_id_metric.loc[:,['id','name', 'status','categories', 'send_at', 'created_at', 'updated_at', 'aggregation',
                                   'bounce_drops', 'bounces', 'clicks', 'delivered', 'invalid_emails', 'opens', 
                                   'requests', 'spam_report_drops', 'spam_reports','unique_clicks', 'unique_opens', 'unsubscribes',
                                   'is_abtest','abtest','ab_phase', 'ab_variation']]
# day_id_metric.to_csv('single send daily statistics.csv')

In [6]:
day_id_metric.categories=day_id_metric.categories.apply(lambda x:x[0] if x != [] else 'No category')

# summary for each singlesend 

In [7]:
params={'page_size': 50,'aggregated_by': 'day'}
response = sg.client.marketing.stats.singlesends.get(query_params=params
)
stats_sum=pd.DataFrame.from_dict(response.to_dict['results'])
metric_df_sum=pd.DataFrame()
for i in range(len(stats_sum)):
    tmp=stats_sum['stats'][i]
    metric_df_sum=metric_df_sum.append(tmp,ignore_index=True)
sum_metric=pd.concat([stats_sum,metric_df_sum],axis=1)
sum_metric.head()

,ab_phase,ab_variation,aggregation,stats,id,bounce_drops,bounces,clicks,delivered,invalid_emails,opens,requests,spam_report_drops,spam_reports,unique_clicks,unique_opens,unsubscribes
0,all,all,total,"{'bounce_drops': 0, 'bounces': 5, 'clicks': 0,...",01c8fb73-b510-11ec-84f5-5e3159b855ec,0.0,5.0,0.0,144.0,0.0,133.0,149.0,0.0,0.0,0.0,58.0,0.0
1,all,all,total,"{'bounce_drops': 4705, 'bounces': 2359, 'click...",042d3e74-910c-11ec-b594-d6b0bb6e2143,4705.0,2359.0,3087.0,25882.0,20.0,8397.0,33129.0,163.0,1.0,1330.0,5093.0,74.0
2,all,all,total,"{'bounce_drops': 101, 'bounces': 66, 'clicks':...",09fefd66-cfd1-11ec-ac99-eacb4a1f4656,101.0,66.0,23.0,400.0,0.0,81.0,567.0,0.0,0.0,3.0,64.0,0.0
3,all,all,total,"{'bounce_drops': 0, 'bounces': 0, 'clicks': 1,...",0dcc8fb2-5172-11ec-9d98-fa15bf9627d2,0.0,0.0,1.0,1.0,0.0,3.0,1.0,0.0,0.0,1.0,1.0,0.0
4,all,all,total,"{'bounce_drops': 5019, 'bounces': 2867, 'click...",0df5f438-b1ed-11ec-94a4-02e777371658,5019.0,2867.0,8150.0,24701.0,20.0,7047.0,32784.0,177.0,1.0,1189.0,4717.0,125.0


In [8]:
sum_metric=pd.merge(sum_metric,single_send_list,on='id',how='left')
sum_metric.columns
sum_metric=sum_metric.loc[:,['id','name', 'status','categories', 'send_at', 'created_at', 'updated_at', 'aggregation',
                                   'bounce_drops', 'bounces', 'clicks', 'delivered', 'invalid_emails', 'opens', 
                                   'requests', 'spam_report_drops', 'spam_reports','unique_clicks', 'unique_opens', 'unsubscribes',
                                   'is_abtest','abtest','ab_phase', 'ab_variation']]
sum_metric.head()

,id,name,status,categories,send_at,created_at,updated_at,aggregation,bounce_drops,bounces,...,requests,spam_report_drops,spam_reports,unique_clicks,unique_opens,unsubscribes,is_abtest,abtest,ab_phase,ab_variation
0,01c8fb73-b510-11ec-84f5-5e3159b855ec,NaN,NaN,NaN,NaN,NaN,NaN,total,0.0,5.0,...,149.0,0.0,0.0,0.0,58.0,0.0,NaN,NaN,all,all
1,042d3e74-910c-11ec-b594-d6b0bb6e2143,02182022-PI-Opportunity,triggered,[],2022-02-18T23:09:47Z,2022-02-18T22:42:14Z,2022-02-18T23:10:08Z,total,4705.0,2359.0,...,33129.0,163.0,1.0,1330.0,5093.0,74.0,False,None,all,all
2,09fefd66-cfd1-11ec-ac99-eacb4a1f4656,NaN,NaN,NaN,NaN,NaN,NaN,total,101.0,66.0,...,567.0,0.0,0.0,3.0,64.0,0.0,NaN,NaN,all,all
3,0dcc8fb2-5172-11ec-9d98-fa15bf9627d2,NaN,NaN,NaN,NaN,NaN,NaN,total,0.0,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,all,all
4,0df5f438-b1ed-11ec-94a4-02e777371658,NaN,NaN,NaN,NaN,NaN,NaN,total,5019.0,2867.0,...,32784.0,177.0,1.0,1189.0,4717.0,125.0,NaN,NaN,all,all


# add two levels of categories 

In [9]:
#manually change single send names
day_id_metric.replace({'name':{'02152022-newsletter':'02152022-atscale-newsletter',
                              'Newsletter_12142021':'aws_Newsletter_12142021',
                              '02012022-Partnership_p':'02012022-nft_competition'}},inplace=True)

In [10]:
for i in range(len(day_id_metric)):
    if re.findall(r'(aws)|(atscale)|(partner)|(kaleidoscope)',day_id_metric.loc[i,'name'].lower()):
        day_id_metric.loc[i,'cate_A']='partner'
        day_id_metric.loc[i,'cate_B']=re.findall(r'aws|atscale|partner|kaleidoscope',day_id_metric.loc[i,'name'].lower())[0]
        if bool(re.findall(r'\d*-aws-(newsletter2)',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='second'
        elif bool(re.findall(r'\d*-aws-(newsletter1?)',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='first'
        if bool(re.findall(r'\d*-atscale-(e-?blast2)',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='second'
        elif bool(re.findall(r'\d*-atscale-(e-?blast1?)',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='first'
    elif re.findall(r'(competition)|(d.c)',day_id_metric.loc[i,'name'].lower()):
        day_id_metric.loc[i,'cate_A']='competition'
        if re.findall(r'(nft_*competition)',day_id_metric.loc[i,'name'].lower()):
            day_id_metric.loc[i,'cate_B']='Creating a NFT value Dashboard'
        elif re.findall(r'(d.c\d*q*\d*)',day_id_metric.loc[i,'name'].lower()):
            day_id_metric.loc[i,'cate_B']=re.findall(r'(d.c\d*q*\d*)',day_id_metric.loc[i,'name'].lower())[0].upper()
        if bool(re.findall(r'.*beginning.*',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='Beginning'
        elif bool(re.findall(r'.*welcom.*webinar.*',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='Welcome Webinar'
        elif bool(re.findall(r'.*training.*webinar.*',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='Training Webinar'
        elif bool(re.findall(r'.*q&?a.*',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='Q&A Webinar'
        elif bool(re.findall(r'.*ceremony.*',day_id_metric.loc[i,'name'].lower())):
            day_id_metric.loc[i,'cate_C']='Ceremony'
    elif re.findall(r'(webinar)|(.*im.*data.*)',day_id_metric.loc[i,'name'].lower()):
        day_id_metric.loc[i,'cate_A']='webinar'
        if re.findall(r'nftsandip',day_id_metric.loc[i,'name'].lower()):
            day_id_metric.loc[i,'cate_B']='NFTs and IP Monetization'
        elif re.findall(r'.*im.*data.*',day_id_metric.loc[i,'name'].lower()):
            day_id_metric.loc[i,'cate_B']='IM Data' 
        else:
            day_id_metric.loc[i,'cate_B']='webinar'
    elif re.findall(r'upskill|pi',day_id_metric.loc[i,'name'].lower()):
        day_id_metric.loc[i,'cate_A']='program'
        day_id_metric.loc[i,'cate_B']=' '.join(re.findall(r'\d*-(\w*)',day_id_metric.loc[i,'name'].lower()))
    elif re.findall(r'nft.*marketplace',day_id_metric.loc[i,'name'].lower()):
        day_id_metric.loc[i,'cate_A']='nft marketplace'
        day_id_metric.loc[i,'cate_B']='nft marketplace'

In [11]:
day_id_metric.head()

,id,name,status,categories,send_at,created_at,updated_at,aggregation,bounce_drops,bounces,...,unique_clicks,unique_opens,unsubscribes,is_abtest,abtest,ab_phase,ab_variation,cate_A,cate_B,cate_C
0,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,Email Blast - Enhance your customers mobile ex...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,2022-03-23,4328.0,2834.0,...,1049.0,1633.0,23.0,False,None,all,all,partner,aws,NaN
1,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,Email Blast - Enhance your customers mobile ex...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,2022-03-24,0.0,9.0,...,11.0,1721.0,20.0,False,None,all,all,partner,aws,NaN
2,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,Email Blast - Enhance your customers mobile ex...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,2022-03-25,0.0,0.0,...,0.0,175.0,2.0,False,None,all,all,partner,aws,NaN
3,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,Email Blast - Enhance your customers mobile ex...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,2022-03-26,0.0,56.0,...,4.0,53.0,3.0,False,None,all,all,partner,aws,NaN
4,1cddb357-aad2-11ec-8d85-b2a3dfe36627,03232022-AWS-EBlast,triggered,Email Blast - Enhance your customers mobile ex...,2022-03-23T22:02:48Z,2022-03-23T17:53:15Z,2022-03-23T22:03:08Z,2022-03-27,0.0,0.0,...,0.0,48.0,0.0,False,None,all,all,partner,aws,NaN


In [12]:
#manual change
day_id_metric.replace({'cate_B':{'DCCQ4':'Creating a NFT value Dashboard',
                                'DSC':'Creating a Dashboard for Responsible Investing using Machine Learning',
                                'DSC2022Q1':'Creating a NFT value Dashboard',
                                'DSCQ3':'Developing an Analytics Dashboard to Improve Restaurant Performance',
                                'DSCQ4':'Creating a Dashboard for Responsible Investing using Machine Learning'
                                }},inplace=True)

In [13]:
print(f'The first level of categories are {day_id_metric.cate_A.unique()}')
print('\n')
print(f'The second level of categories are {day_id_metric.cate_B.unique()}')
print(f'The third level of categories are {day_id_metric.cate_C.unique()}')

The first level of categories are ['partner' 'webinar' 'competition' 'program']


The second level of categories are ['aws' 'kaleidoscope' 'atscale' 'NFTs and IP Monetization'
 'Creating a Dashboard for Responsible Investing using Machine Learning'
 'pi opportunity' 'Creating a NFT value Dashboard' 'upskill covid 19 p'
 'upskill stock_market'
 'Developing an Analytics Dashboard to Improve Restaurant Performance']
The third level of categories are [nan 'second' 'first' 'Ceremony' 'Q&A Webinar' 'Beginning']


# pivot table 

In [14]:
pv_table=pd.pivot_table(day_id_metric,
               index=['id','name','aggregation'],
               values=['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes'],
               aggfunc='sum'
              )
pv_table=pv_table.loc[:,['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes']]
pv_table['open_rates']=round(pv_table['opens']/pv_table['delivered'],4)
pv_table['click_rates']=round(pv_table['clicks']/pv_table['delivered'],4)
pv_table['CTOR']=round(pv_table['unique_clicks']/pv_table['unique_opens'],4)
pv_table.head()

delivered  \
id                                   name                    aggregation              
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18     25861.0   
                                                             2022-02-19        15.0   
                                                             2022-02-20         5.0   
                                                             2022-02-21         1.0   
                                                             2022-02-22         0.0   

                                                                           opens  \
id                                   name                    aggregation           
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18   2111.0   
                                                             2022-02-19   3651.0   
                                                             2022-02-20    717.0   
                                                             2022-02-21    633.0   
                                                             2022-02-22    373.0   

                                                                          clicks  \
id                                   name                    aggregation           
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18   2695.0   
                                                             2022-02-19    214.0   
                                                             2022-02-20     30.0   
                                                             2022-02-21     60.0   
                                                             2022-02-22     24.0   

                                                                          unique_opens  \
id                                   name                    aggregation                 
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18         1595.0   
                                                             2022-02-19         2332.0   
                                                             2022-02-20          301.0   
                                                             2022-02-21          278.0   
                                                             2022-02-22          155.0   

                                                                          unique_clicks  \
id                                   name                    aggregation                  
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18          1212.0   
                                                             2022-02-19            69.0   
                                                             2022-02-20            14.0   
                                                             2022-02-21            11.0   
                                                             2022-02-22             4.0   

                                                                          requests  \
id                                   name                    aggregation             
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18    33129.0   
                                                             2022-02-19        0.0   
                                                             2022-02-20        0.0   
                                                             2022-02-21        0.0   
                                                             2022-02-22        0.0   

                                                                          bounce_drops  \
id                                   name                    aggregation                 
042d3e74-910c-11ec-b594-d6b0bb6e2143 02182022-PI-Opportunity 2022-02-18         4705.0   
                                                             2022-02-19            0.0   
                                                      

In [15]:
activity_duration=pd.pivot_table(day_id_metric,
               index=['cate_A','cate_B','name'],
               values=['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes'],
               aggfunc=np.count_nonzero
              )
activity_duration=activity_duration.loc[:,['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes']]
activity_duration.head()

delivered  \
cate_A      cate_B                                             name                                                   
competition Creating a Dashboard for Responsible Investing ... 01072022-DSC-Beginning-Reminder                  1.0   
                                                               01102022-DSCQ4-welcome-webnair_Reminder          1.0   
                                                               01102022-DSC_f_P                                 4.0   
                                                               01112022-DSCQ4-training-webnair_Reminder1        1.0   
                                                               01112022-DSCQ4-training-webnair_Reminder2        1.0   

                                                                                                          opens  \
cate_A      cate_B                                             name                                               
competition Creating a Dashboard for Responsible Investing ... 01072022-DSC-Beginning-Reminder             13.0   
                                                               01102022-DSCQ4-welcome-webnair_Reminder     12.0   
                                                               01102022-DSC_f_P                            25.0   
                                                               01112022-DSCQ4-training-webnair_Reminder1   13.0   
                                                               01112022-DSCQ4-training-webnair_Reminder2   14.0   

                                                                                                          clicks  \
cate_A      cate_B                                             name                                                
competition Creating a Dashboard for Responsible Investing ... 01072022-DSC-Beginning-Reminder               7.0   
                                                               01102022-DSCQ4-welcome-webnair_Reminder       0.0   
                                                               01102022-DSC_f_P                             14.0   
                                                               01112022-DSCQ4-training-webnair_Reminder1     0.0   
                                                               01112022-DSCQ4-training-webnair_Reminder2     0.0   

                                                                                                          unique_opens  \
cate_A      cate_B                                             name                                                      
competition Creating a Dashboard for Responsible Investing ... 01072022-DSC-Beginning-Reminder                     8.0   
                                                               01102022-DSCQ4-welcome-webnair_Reminder            11.0   
                                                               01102022-DSC_f_P                                   25.0   
                                                               01112022-DSCQ4-training-webnair_Reminder1          13.0   
                                                               01112022-DSCQ4-training-webnair_Reminder2          12.0   

                                                                                                          unique_clicks  \
cate_A      cate_B                                             name                                                       
competition Creating a Dashboard for Responsible Investing ... 01072022-DSC-Beginning-Reminder                      5.0   
                                                               01102022-DSCQ4-welcome-webnair_Reminder              0.0   
                                                               01102022-DSC_f_P                                    12.0   
                                                               01112022-DSCQ4-training-webnair_Reminder1            0.0   
                                                            

# partner_df

In [16]:
partner_df=pd.pivot_table(day_id_metric.loc[day_id_metric['cate_A']=='partner',],
               index=['cate_A','cate_B','name','cate_C'],
               values=['delivered','opens','clicks', 'unique_opens','unique_clicks'],
               aggfunc='sum')

partner_df['open_rates']=round(partner_df['opens']/partner_df['delivered'],4)
partner_df['click_rates']=round(partner_df['clicks']/partner_df['delivered'],4)
partner_df['CTOR']=round(partner_df['unique_clicks']/partner_df['unique_opens'],4)
partner_df.head()

clicks  delivered   opens  \
cate_A  cate_B  name                     cate_C                               
partner atscale 01212022-atscale-eblast  first   10402.0    27084.0  8395.0   
                11022022-atscale-eblast  first   11185.0    25900.0  7179.0   
        aws     03152022-AWS-Newsletter1 first    9678.0    22196.0  7350.0   
                03222022-AWS-Newsletter2 second   7825.0    21620.0  5463.0   

                                                 unique_clicks  unique_opens  \
cate_A  cate_B  name                     cate_C                                
partner atscale 01212022-atscale-eblast  first          1091.0        5125.0   
                11022022-atscale-eblast  first          1204.0        4977.0   
        aws     03152022-AWS-Newsletter1 first          1112.0        5090.0   
                03222022-AWS-Newsletter2 second          921.0        3631.0   

                                                 open_rates  click_rates  \
cate_A  cate_B  name                     cate_C                            
partner atscale 01212022-atscale-eblast  first       0.3100       0.3841   
                11022022-atscale-eblast  first       0.2772       0.4319   
        aws     03152022-AWS-Newsletter1 first       0.3311       0.4360   
                03222022-AWS-Newsletter2 second      0.2527       0.3619   

                                                   CTOR  
cate_A  cate_B  name                     cate_C          
partner atscale 01212022-atscale-eblast  first   0.2129  
                11022022-atscale-eblast  first   0.2419  
        aws     03152022-AWS-Newsletter1 first   0.2185  
                03222022-AWS-Newsletter2 second  0.2536

# process df 

In [17]:
process_df=pd.pivot_table(day_id_metric,
               index=['cate_A','cate_B','cate_C','name'],
               values=['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes'],
                          aggfunc='sum'
              )
process_df=process_df.loc[:,['delivered','opens','clicks', 'unique_opens','unique_clicks', 'requests','bounce_drops', 'bounces' , 'invalid_emails','spam_report_drops', 'spam_reports',  'unsubscribes']]

# Weekly new single send 

In [18]:
weekly=day_id_metric.head(10).copy()
weekly['send_at']=weekly['send_at'].apply(lambda x:datetime.datetime.strptime(str(x),'%Y-%m-%dT%H:%M:%SZ'))
tmd_date=datetime.datetime.strptime(end_date,'%Y-%m-%d')-datetime.timedelta(days=4)
weekly_single_send=weekly.loc[weekly['send_at']>tmd_date,['name','cate_A','cate_B','cate_C','aggregation','delivered','unique_clicks','unique_opens','clicks','opens']]
weekly_single_send['open_rates']=round(weekly_single_send['opens']/weekly_single_send['delivered'],4)
weekly_single_send['click_rates']=round(weekly_single_send['clicks']/weekly_single_send['delivered'],4)
weekly_single_send['CTOR']=round(weekly_single_send['unique_clicks']/weekly_single_send['unique_opens'],4)

# write into a template 

In [19]:
import xlsxwriter
writer=pd.ExcelWriter(f"/Users/georgiaxiao/Desktop/RMDS/Tianxiao-Analytics/single send template/daily statistic dashboard_{end_date}.xlsx",engine='xlsxwriter')
weekly_single_send.to_excel(writer,sheet_name='weekly updated',index=False)
activity_duration.to_excel(writer,sheet_name='duration')
partner_df.to_excel(writer,sheet_name='partner')
pv_table.to_excel(writer,sheet_name='time_variant')
process_df.to_excel(writer, sheet_name='process')
sum_metric.to_excel(writer,sheet_name='summary',index=False)
day_id_metric.to_excel(writer,sheet_name='daily data',index=False)
writer.save()

# Email Marketing updating

In [20]:
sum_metric.columns

Index(['id', 'name', 'status', 'categories', 'send_at', 'created_at',
       'updated_at', 'aggregation', 'bounce_drops', 'bounces', 'clicks',
       'delivered', 'invalid_emails', 'opens', 'requests', 'spam_report_drops',
       'spam_reports', 'unique_clicks', 'unique_opens', 'unsubscribes',
       'is_abtest', 'abtest', 'ab_phase', 'ab_variation'],
      dtype='object')

In [21]:
updating_df=sum_metric.merge(day_id_metric.loc[:,['id','cate_B']])
updating_df['Open Rate']=round(updating_df['unique_opens']/updating_df['delivered']*100,2)
updating_df['Click Rate']=round(updating_df['unique_clicks']/updating_df['delivered']*100,2)
updating_df['Click Through Rate']=round(updating_df['unique_clicks']/updating_df['unique_opens']*100,2)
updating_df=updating_df.loc[:,['name','cate_B','Open Rate','Click Rate','Click Through Rate','send_at']].drop_duplicates()
updating_df['send_at']=updating_df['send_at'].apply(lambda x:(datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%SZ')))
updating_df.sort_values(by='send_at',ascending=False, inplace=True)
updating_df['send_at']=updating_df['send_at'].apply(lambda x:(x.strftime('%m/%d/%y %H:%M')))
updating_df.columns=['Email Name','Campaign','Open Rate','Click Rate','Click Through Rate','Send Date(Your time zone)']
updating_df.to_clipboard(index=False)